In [ ]:
# Micro-SAM Run Inference on OMERO Data

Example notebook to try the finetuned micro_sam model on OMERO data
## 1. Setup 
Run these cells to import all required packages and functions
# Import the omero login widget
from omero_annotate_ai import (
    create_omero_connection_widget,
)

# Additional imports
import datetime
from pathlib import Path
import torch
import ezomero
import stackview
import numpy as np

from micro_sam.automatic_segmentation import get_predictor_and_segmenter, automatic_instance_segmentation

print(f"Device available: {'CUDA' if torch.cuda.is_available() else 'CPU'}")


## 2. OMERO Connection

In [ ]:
# Create and display OMERO connection widget
conn_widget = create_omero_connection_widget()
conn_widget.display()
# After setting up the setting we need to setup the OMERO connection
conn = conn_widget.get_connection()

if conn is None:
    raise ConnectionError("No OMERO connection established.")

print(f"Connected to OMERO as: {conn.getUser().getName()}")

## 3. Data Selection

In [ ]:
# Select image and a channel, timepoint z-slice (currently only 2D)
image_id = 277
channel = 0
time_point = 66
z_slice = 0
#optional path to custom trained model
#checkpoint_path = r'micro_sam_training_20250829_135659_final.pt'

In [ ]:
#get image from OMERO
image = conn.getObject("Image", image_id)
__,pixels = ezomero.get_image(conn, image_id, start_coords=[0,0,z_slice,channel,time_point], axis_lengths=[image.getSizeX(),image.getSizeY(),1,1,1])
#Run prediction
labels = run_automatic_instance_segmentation(np.squeeze(pixels), checkpoint_path, model_type="vit_b_lm", device=None)

In [ ]:
#Here you can see your image with labels overlayed
stackview.curtain(np.squeeze(pixels), labels, alpha=0.5, continuous_update=True,zoom_factor=0.5)

An example how you can run the prediction on a stack, a time series in this example


In [ ]:
# Select image
image_id = 277
channel = 0
time_point = 0
time_point_length = 30
z_slice = 0

#get image from OMERO
image = conn.getObject("Image", image_id)
__,pixels = ezomero.get_image(conn, image_id, start_coords=[0,0,z_slice,channel,time_point], axis_lengths=[image.getSizeX(),image.getSizeY(),1,1,time_point_length])
#Run prediction
labels = []
for i in range(np.shape(pixels)[0]):
    labels.append(run_automatic_instance_segmentation(np.squeeze(pixels[i]), checkpoint_path, model_type="vit_b_lm", device=None))
#Now you can also slice through the images
stackview.curtain(np.squeeze(pixels), labels, alpha=0.5, continuous_update=True,zoom_factor=0.5)